In [ ]:
# !wget https://huggingface.co/datasets/malaysia-ai/starcoderdata-sample/resolve/main/scala.jsonl

In [4]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
from glob import glob

In [5]:
ls = []

with open('scala.jsonl') as fopen:
    for l in tqdm(fopen):
        try:
            l = json.loads(l)
            if len(l['content'].split()) > 5e3:
                continue
            ls.append(l['content'])
        except:
            pass
len(ls)

100000it [00:04, 21508.06it/s]


99472

In [6]:
client = InferenceClient(
    "", timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [7]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [8]:
!mkdir mixtral-scala
# !rm -rf mixtral-scala/*.json

In [9]:
def answer(q, i):
    filename = f'mixtral-scala/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
    # for _ in range(5):
        try:
            
            prompt = f"""
Please gain inspiration from the following random code snippet to create a high-quality JavaScript problem. Present your output in two distinct sections: [Problem Description] and [Solution].

JavaScript snippet for inspiration:
```
{q}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included but do not mention code snippet above.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except Exception as e:
            if 'tokens + `max_new_tokens`' in str(e):
                with open(filename, 'w') as fopen:
                    json.dump(False, fopen)
                break
            pass

In [10]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [11]:
from threading import Thread
from queue import Queue

queue = Queue()
questions = ls
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [12]:
answer(*urls[0])

In [13]:
with open('mixtral-scala/0.json') as fopen:
    print(json.loads(fopen.read()))

 **[Problem Description]**

Waves Explorer is a tool for exploring and querying data in the Waves blockchain. This problem involves implementing a simplified version of the Waves Explorer for a subset of its functionalities. You are required to implement an in-memory blockchain data store, handle user queries, and output the appropriate information.

You are given a list of keys (key prefixes) in the form of an array called `keys`. This array consists of various blockchain data categories, e.g., `version`, `height`, `score`, `block-at-height`, `height-of`, etc. Each key prefix represents a specific piece of information in the Waves blockchain.

The input to your solution would be a CLI (Command Line Interface) with the following arguments:

- Config filename: The path to the configuration file containing data directory and other settings (not used in this problem).
- Flag: A single character flag representing a query type (B, O, A, AC, AD, AA, S).
- Arguments for the flag: One or more 

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 12%|███████████▉                                                                                      | 12098/99472 [13:32:03<67:12:03,  2.77s/it]